# Data Preparation des Legend-Sheets

In [2]:
import pandas as pd

In [3]:
# Einlesen des gesamten Legend-Sheets ohne Header
file_path = "original_data/Results.xlsx"

1. Sheet einlesen als Dataframe
2. Nach Tabellennamen in ersten Zeile Suchen
3. Einzelne Tabellen von links nach rechts erstellen, NaN-Spalte ist Abbruchkriterium 

In [5]:
def split_legend_sheet(df):
    tables = {}  # Dictionary zur Speicherung der Tabellen
    table_start_col = None  # Startspalte der aktuellen Tabelle
    table_name = None  # Name der aktuellen Tabelle
    
    # Durchlaufen der Spalten
    for col in df.columns:
        if pd.notna(df.iloc[1, col]):  # Tabellenname gefunden in Zeile 2
            if table_start_col is not None:
                # Speichere die vorherige Tabelle
                data_start = 4
                table_data = df.iloc[data_start:, table_start_col:col]  # Erstelle DataFrame
                table_data = table_data.dropna(axis=1, how='all').reset_index(drop=True)  # Entferne leere Spalten
                table_data.columns = df.iloc[data_start - 1, table_start_col:col].dropna().values  # Header setzen
                tables[table_name] = table_data
            # Start einer neuen Tabelle
            table_start_col = col
            table_name = df.iloc[1, col]
    
    # Letzte Tabelle speichern
    if table_start_col is not None and table_name:
        data_start = 4
        table_data = df.iloc[data_start:, table_start_col:]  # Erstelle DataFrame
        table_data = table_data.dropna(axis=1, how='all').reset_index(drop=True)  # Entferne leere Spalten
        table_data.columns = df.iloc[data_start - 1, table_start_col:].dropna().values  # Header setzen
        tables[table_name] = table_data
    
    return tables

In [6]:
def _combine_sample_names(df, column_index, main_substring="PE", new_column_name=None):
    """
    Kombiniert Hauptnamen mit den Unterzeilen und löscht Zeilen ohne Unterstrich im angegebenen Feld.

    Parameter:
    df (pd.DataFrame): Das DataFrame, das bearbeitet werden soll.
    column_index (int): Der Index der Spalte, die angepasst und gefiltert wird.
    main_substring (str): Der Substring, der Hauptnamen identifiziert.
    new_column_name (str, optional): Der neue Name der Spalte, falls die Spalte umbenannt werden soll.

    Rückgabe:
    pd.DataFrame: Ein gefiltertes DataFrame ohne Hauptnamenzeilen.
    """
    # 0. Zeilen löschen, wo der Wert in der angegebenen Spalte NaN ist
    df = df[df.iloc[:, column_index].notna()]

    current_main_name = None
    new_sample_names = []

    # 1. Hauptnamen mit Unterwerten kombinieren
    for value in df.iloc[:, column_index]:
        if isinstance(value, str) and main_substring in value:  # Hauptname erkannt
            current_main_name = value
            new_sample_names.append(value)  # Hauptzeile bleibt gleich
        elif current_main_name:  # Unterzeilen kombinieren mit Hauptname
            new_name = f"{current_main_name}_{value}"
            new_sample_names.append(new_name)
        else:  # Für NaN oder sonstige Fälle
            new_sample_names.append(value)

    # Kopie erstellen und aktualisieren, um Warning zu vermeiden
    df_copy = df.copy()
    df_copy.iloc[:, column_index] = new_sample_names

    # Spalte umbenennen, falls new_column_name angegeben ist
    if new_column_name:
        df_copy.rename(columns={df_copy.columns[column_index]: new_column_name}, inplace=True)

    # 2. Zeilen löschen, die keinen Unterstrich enthalten
    df_filtered = df_copy[df_copy.iloc[:, column_index].str.contains("_", na=False)]

    return df_filtered

In [7]:
def concat_tables(tables_dict):
    """
    Transformiert alle Tabellen im Dictionary und kombiniert sie in ein einziges DataFrame.
    Führt folgende Schritte aus:

    1. Teilt die Legende im Tabellenblatt.
    2. Kombiniert 'Sample Name' mit Unterwerten.
    3. Fügt eine neue Spalte 'Aging Level' hinzu, die den Tabellennamen (Key) aus dem Dictionary enthält.

    Parameter:
    tables_dict (dict): Dictionary mit Tabellen (DataFrames).

    Rückgabe:
    pd.DataFrame: Ein kombiniertes DataFrame mit einer zusätzlichen 'Aging Level'-Spalte.
    """
    combined_df = []

    for table_name, table_df in tables_dict.items():

        # Schritt 1: Sample Names kombinieren
        table_df = _combine_sample_names(table_df, 0, main_substring="-", new_column_name = "Sample Name")

        # Schritt 2: 'Aging Level'-Spalte hinzufügen
        table_df['Aging Level'] = table_name

        # DataFrame zur kombinierten Liste hinzufügen
        combined_df.append(table_df)

    # Alle DataFrames vertikal kombinieren
    return pd.concat(combined_df, ignore_index=True)

## Pristine LDPE and PET

In [9]:
df_legend_raw = pd.read_excel(file_path, sheet_name = "Legend", header=None)
df_legend_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Pristine LDPE and PET,NaN,NaN,First UV-cycle (4 weeks irradiation),NaN,NaN,Second UV-cycle (8 weeks irradiation),NaN,NaN,...,NaN,NaN,First EBT,NaN,NaN,Second EBT,NaN,NaN,Third EBT,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0-PE,Name of Original DSC Measurement,NaN,1-UV-PE,Name of Original DSC Measurement,NaN,2-UV-PE,Name of Original DSC Measurement,NaN,...,Name of Original DSC Measurement,NaN,1-EBT-PE,Name of Original DSC Measurement,NaN,2-EBT-PE,Name of Original DSC Measurement,NaN,3-EBT-PE,Name of Original DSC Measurement
4,NaN,0-PE-1,NaN,NaN,1-UV-PE-1,NaN,NaN,2-UV-PE-1,NaN,NaN,...,NaN,NaN,1-EBT-PE-1,NaN,NaN,2-EBT-PE-1,NaN,NaN,3-EBT-PE-1,NaN


In [10]:
# Aufteilen der Tabellen nach Spalten
tables_dict = split_legend_sheet(df_legend_raw)
df_pristine = tables_dict.get("Pristine LDPE and PET")
df_pristine

,0-PE,Name of Original DSC Measurement
0,0-PE-1,NaN
1,1,0-PE-6
2,2,0-PE-7
3,3,0-PE-8
4,4,0-PE-9
...,...,...
63,NaN,NaN
64,NaN,NaN
65,NaN,NaN
66,NaN,NaN


In [11]:
df_pristine_edit = _combine_sample_names(df_pristine, 0, main_substring="PE", new_column_name="Sample Name")
df_pristine_edit.head(10)

,Sample Name,Name of Original DSC Measurement
1,0-PE-1_1,0-PE-6
2,0-PE-1_2,0-PE-7
3,0-PE-1_3,0-PE-8
4,0-PE-1_4,0-PE-9
5,0-PE-1_5,0-PE-10
7,0-PE-2_1,0-PE-11
8,0-PE-2_2,0-PE-12
9,0-PE-2_3,0-PE-13
10,0-PE-2_4,0-PE-14
11,0-PE-2_5,0-PE-15


## Alle

In [13]:
df_legend_raw = pd.read_excel(file_path, sheet_name = "Legend", header=None)

legend_dict = split_legend_sheet(df_legend_raw)

In [14]:
df_legend = concat_tables(tables_dict)
df_legend

,Sample Name,Name of Original DSC Measurement,Aging Level
0,0-PE-1_1,0-PE-6,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,Pristine LDPE and PET
...,...,...,...
289,3-EBT-PET-4_1,eb-PET-62,Third EBT
290,3-EBT-PET-4_2,eb-PET-63,Third EBT
291,3-EBT-PET-4_3,eb-PET-64,Third EBT
292,3-EBT-PET-4_4,eb-PET-65,Third EBT


In [15]:
df_legend.to_csv("data/01_Legend.csv", index=False)

In [16]:
df_legend["Aging Level"].value_counts()

Aging Level
Third UV-cycle (12 weks irradiation)     55
Pristine LDPE and PET                    40
First UV-cycle (4 weeks irradiation)     40
Second UV-cycle (8 weeks irradiation)    40
First EBT                                40
Third EBT                                40
Second EBT                               39
Name: count, dtype: int64